In [137]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import TargetEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from amplpy import AMPL, ampl_notebook
from joblib import load, dump
import sys
sys.path.append("../src")
from utils import cost_prediction

import warnings
warnings.filterwarnings("ignore")
pd.set_option("future.no_silent_downcasting", True) # Prevent silent data type changes during operations for future compatibility

Instantiate the AMPL object

In [138]:
ampl = ampl_notebook(
    modules=["highs", "coin"], # solver to be used
    license_uuid="d2b35988-195b-44a4-bca2-fd80a770586f" # license key
)

Licensed to AMPL Community Edition License for <hchoi0309@gmail.com>.


In [139]:
%%ampl_eval
reset;

Define the model

In [140]:
%%ampl_eval
# Sets
set ROUTES;
set AIRPLANE_TYPES;

# Parameters
param a {ROUTES};          # Base fare price coefficient
param b {ROUTES};          # Price elasticity coefficient
param y {ROUTES};          # Yearly effect coefficient
param c {ROUTES};          # Cost per seat
param N {ROUTES};          # Predicted daily passengers
param C {AIRPLANE_TYPES};  # Capacity of each airplane type
param R {AIRPLANE_TYPES};  # Total airplanes of each type
param t;                   # Target market share (fraction)
param year;                # Optimization year

# Variables
var x {ROUTES} > 0, integer;      # Daily number of seats sold
var r {ROUTES, AIRPLANE_TYPES} >= 0, integer;  # Number of airplanes assigned per route

# Objective
maximize Profit:
    sum {i in ROUTES} (
        x[i] * (a[i] * exp(-b[i] * x[i] + y[i] * year) - c[i])
    );

# Constraints
s.t. MarketShare {i in ROUTES}:
    x[i] <= N[i];

s.t. MinCapacity {i in ROUTES}:
    x[i] >= sum {k in AIRPLANE_TYPES} 0.7 * C[k] * r[i, k];

s.t. MaxCapacity {i in ROUTES}:
    x[i] <= sum {k in AIRPLANE_TYPES} C[k] * r[i, k];

s.t. FleetLimit {k in AIRPLANE_TYPES}:
    sum {i in ROUTES} r[i, k] <= 3 * R[k];

	line 18 offset 578
	cannot enforce strict bound
	context:  var x {ROUTES} >  >>> 0, <<<  integer;      # Daily number of seats sold


In [141]:
df = pd.read_csv("../data/FINAL_FINAL.csv")
display(df.head())

,airport_1,airport_2,year_coeff,nsmiles,a,b,airport_1_2,city_1,state_1,airport_2.1,city_2,state_2,population_1,density_1,lat_1,lon_1,population_2,density_2,lat_2,lon_2
0,AUS,IAD,-0.009039,1342,353.079573,0.000288,AUS_IAD,Austin,TX,IAD,Washington,DC,1905945,1154.1,30.3005,-97.7522,5116378,4235.7,38.9047,-77.0163
1,BNA,IAD,0.013684,587,300.216095,0.000731,BNA_IAD,Nashville,TN,IAD,Washington,DC,1177657,555.4,36.1715,-86.7842,5116378,4235.7,38.9047,-77.0163
2,BOS,GRR,0.034155,740,391.965759,0.001834,BOS_GRR,Boston,MA,GRR,Grand Rapids,MI,4328315,5319.0,42.3188,-71.0852,609023,1708.2,42.9619,-85.6562
3,BOS,IND,-0.000761,818,253.127775,0.000074,BOS_IND,Boston,MA,IND,Indianapolis,IN,4328315,5319.0,42.3188,-71.0852,1729849,941.8,39.7771,-86.1458
4,BOS,ORF,0.010823,487,215.865696,0.000175,BOS_ORF,Boston,MA,ORF,Norfolk,VA,4328315,5319.0,42.3188,-71.0852,236973,1717.4,36.8945,-76.2590


In [142]:
df["year"] = 2019
df["quarter"] = 4
df = df.rename({"nsmiles": "distance"}, axis=1)
df = df.drop(df[df["airport_1_2"] == "LAX_HPN"].index)
display(df.head())

,airport_1,airport_2,year_coeff,distance,a,b,airport_1_2,city_1,state_1,airport_2.1,...,population_1,density_1,lat_1,lon_1,population_2,density_2,lat_2,lon_2,year,quarter
0,AUS,IAD,-0.009039,1342,353.079573,0.000288,AUS_IAD,Austin,TX,IAD,...,1905945,1154.1,30.3005,-97.7522,5116378,4235.7,38.9047,-77.0163,2019,4
1,BNA,IAD,0.013684,587,300.216095,0.000731,BNA_IAD,Nashville,TN,IAD,...,1177657,555.4,36.1715,-86.7842,5116378,4235.7,38.9047,-77.0163,2019,4
2,BOS,GRR,0.034155,740,391.965759,0.001834,BOS_GRR,Boston,MA,GRR,...,4328315,5319.0,42.3188,-71.0852,609023,1708.2,42.9619,-85.6562,2019,4
3,BOS,IND,-0.000761,818,253.127775,0.000074,BOS_IND,Boston,MA,IND,...,4328315,5319.0,42.3188,-71.0852,1729849,941.8,39.7771,-86.1458,2019,4
4,BOS,ORF,0.010823,487,215.865696,0.000175,BOS_ORF,Boston,MA,ORF,...,4328315,5319.0,42.3188,-71.0852,236973,1717.4,36.8945,-76.2590,2019,4


In [143]:
df["cost"] = cost_prediction(df, 2019)
display(df.head())

,airport_1,airport_2,year_coeff,distance,a,b,airport_1_2,city_1,state_1,airport_2.1,...,density_1,lat_1,lon_1,population_2,density_2,lat_2,lon_2,year,quarter,cost
0,AUS,IAD,-0.009039,1342,353.079573,0.000288,AUS_IAD,Austin,TX,IAD,...,1154.1,30.3005,-97.7522,5116378,4235.7,38.9047,-77.0163,2019,4,234.94394
1,BNA,IAD,0.013684,587,300.216095,0.000731,BNA_IAD,Nashville,TN,IAD,...,555.4,36.1715,-86.7842,5116378,4235.7,38.9047,-77.0163,2019,4,102.76609
2,BOS,GRR,0.034155,740,391.965759,0.001834,BOS_GRR,Boston,MA,GRR,...,5319.0,42.3188,-71.0852,609023,1708.2,42.9619,-85.6562,2019,4,129.55180
3,BOS,IND,-0.000761,818,253.127775,0.000074,BOS_IND,Boston,MA,IND,...,5319.0,42.3188,-71.0852,1729849,941.8,39.7771,-86.1458,2019,4,143.20726
4,BOS,ORF,0.010823,487,215.865696,0.000175,BOS_ORF,Boston,MA,ORF,...,5319.0,42.3188,-71.0852,236973,1717.4,36.8945,-76.2590,2019,4,85.25909


In [144]:
preprocessing_pipeline = load("../models/preprocessing_pipeline.joblib")
feature_names = load("../models/feature_names.joblib")

df_transformed = preprocessing_pipeline.transform(df.drop(["year_coeff", "a", "b", "airport_1_2", "airport_2.1", "cost"], axis=1))
df_transformed = pd.DataFrame(df_transformed, columns=feature_names)
display(df_transformed.head())

,quarter_1,quarter_2,quarter_3,quarter_4,city_1,city_2,airport_1,airport_2,state_1,state_2,year,distance,population_1,density_1,population_2,density_2,lat_1,lon_1,lat_2,lon_2
0,-0.592028,-0.571486,-0.576259,1.755523,-0.042234,-0.013962,-0.032676,-0.502082,-0.099588,-0.003964,1.204085,0.216246,-0.694892,-0.715648,-0.172653,0.043854,-1.699120,-0.522722,0.129862,0.839686
1,-0.592028,-0.571486,-0.576259,1.755523,-0.162528,-0.013962,-0.117519,-0.502082,-1.044005,-0.003964,1.204085,-0.857693,-0.826930,-0.916193,-0.172653,0.043854,-0.464798,0.186303,0.129862,0.839686
2,-0.592028,-0.571486,-0.576259,1.755523,-0.050861,-1.271365,1.496511,-0.855942,-0.056457,0.950370,1.204085,-0.640060,-0.255718,0.679458,-0.904045,-0.688692,0.827613,1.201164,0.989342,0.384514
3,-0.592028,-0.571486,-0.576259,1.755523,-0.050861,-0.751074,1.496511,-0.507130,-0.056457,-1.230771,1.204085,-0.529110,-0.255718,0.679458,-0.722173,-0.910818,0.827613,1.201164,0.314672,0.358721
4,-0.592028,-0.571486,-0.576259,1.755523,-0.050861,-1.717950,1.496511,-0.952062,-0.056457,-2.422257,1.204085,-0.999937,-0.255718,0.679458,-0.964416,-0.686026,0.827613,1.201164,-0.295980,0.879582


In [145]:
demand_model = load("../models/demand_prediction_model.joblib")

df["predicted_passengers"] = np.abs(demand_model.predict(df_transformed))
df = df[["year_coeff", "a", "b", "airport_1_2", "cost", "predicted_passengers"]]
display(df.head())

,year_coeff,a,b,airport_1_2,cost,predicted_passengers
0,-0.009039,353.079573,0.000288,AUS_IAD,234.94394,323.784943
1,0.013684,300.216095,0.000731,BNA_IAD,102.76609,220.126175
2,0.034155,391.965759,0.001834,BOS_GRR,129.55180,151.138489
3,-0.000761,253.127775,0.000074,BOS_IND,143.20726,512.026611
4,0.010823,215.865696,0.000175,BOS_ORF,85.25909,243.720016


In [146]:
fleet_df = pd.read_csv("../data/allegiant_fleet.csv")
display(fleet_df)

,Aircraft Type,Number of Aircraft,Seats
0,Airbus A319-100,34,156
1,Airbus A320-200 A,14,177
2,Airbus A320-200 B,78,186
3,Boeing 737 MAX 200,2,190


Define the problem

In [147]:
# 2. Load the data into AMPL
# Assuming df and fleet_df are your dataframes containing the necessary data
ampl.set["ROUTES"] = df["airport_1_2"].unique()
ampl.set["AIRPLANE_TYPES"] = fleet_df["Aircraft Type"]

# Load parameters
ampl.param["a"] = {row["airport_1_2"]: row["a"] for _, row in df.drop_duplicates("airport_1_2").iterrows()}
ampl.param["b"] = {row["airport_1_2"]: row["b"] for _, row in df.drop_duplicates("airport_1_2").iterrows()}
ampl.param["y"] = {row["airport_1_2"]: row["year_coeff"] for _, row in df.drop_duplicates("airport_1_2").iterrows()}
ampl.param["c"] = {row["airport_1_2"]: row["cost"] for _, row in df.drop_duplicates("airport_1_2").iterrows()}
ampl.param["N"] = {row["airport_1_2"]: row["predicted_passengers"] for _, row in df.drop_duplicates("airport_1_2").iterrows()}

ampl.param["C"] = {row["Aircraft Type"]: row["Seats"] for _, row in fleet_df.iterrows()}
ampl.param["R"] = {row["Aircraft Type"]: row["Number of Aircraft"] for _, row in fleet_df.iterrows()}

ampl.param["t"] = 0.8  # Target market share (80%)
ampl.param["year"] = 2024  # Current year

# 3. Set solver and solve
ampl.option["solver"] = "ipopt"  # Use ipopt for nonlinear problems
ampl.solve()

# 4. Retrieve and display results
try:
    # Get results
    x_values = ampl.getVariable("x").getValues().toPandas()
    r_values = ampl.getVariable("r").getValues().toPandas()
    profit = ampl.getObjective("Profit").value()

    print("Optimal seat allocation:")
    print(x_values)
    print("\nAircraft assignment:")
    print(r_values)
    print(f"\nTotal profit: ${profit:,.2f}")

except Exception as e:
    print("Error solving the model:")
    print(e)
    
    # Debug information
    print("\nParameter values:")
    print("Routes:", list(ampl.set["ROUTES"]))
    print("N values:", ampl.param["N"].getValues().toPandas())
    print("a values:", ampl.param["a"].getValues().toPandas())

Ipopt 3.12.13: 
==> Warning: Treating 134 binary and 817 integer variables as continous.


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     2661
Number of nonzeros in Lagrangian Hessian.............:      140

Total number of variables............................:      951
                     variables with only lower bounds:        0
                variables with lower and upper bounds:   

In [116]:
ampl.set["ROUTES"] = df["airport_1_2"].drop_duplicates()
ampl.param["a"] = df.set_index("airport_1_2")["a"]
ampl.param["b"] = df.set_index("airport_1_2")["b"]
ampl.param["y"] = df.set_index("airport_1_2")["year_coeff"]
ampl.param["c"] = df.set_index("airport_1_2")["cost"]
ampl.param["N"] = df.set_index("airport_1_2")["predicted_passengers"]

ampl.set["AIRPLANE_TYPES"] = fleet_df["Aircraft Type"]
ampl.param["C"] = fleet_df.groupby("Aircraft Type")["Seats"].first()
ampl.param["R"] = fleet_df.groupby("Aircraft Type")["Number of Aircraft"].first()

ampl.param["t"] = 0.8
ampl.param["year"] = 2019

In [117]:
%%ampl_eval
option solver ipopt
solve;

In [118]:
# Retrieve results
x = ampl.getVariable("x").getValues().toPandas()
r = ampl.getVariable("r").getValues().toPandas()
profit = ampl.getObjective("Profit").value()

# Display results
print("Optimal Seat Sales (x):")
print(x)

print("\nAirplane Assignments (r):")
print(r)

print(f"\nTotal Profit: {profit}")

Optimal Seat Sales (x):
         x.val
AUS_BOS      0
AUS_BUR      0
AUS_BWI      0
AUS_CLE      0
AUS_CMH      0
...        ...
PHX_TUL      0
SNA_GEG      0
SNA_MFR      0
SRQ_IAD      0
VPS_BWI      0

[202 rows x 1 columns]

Airplane Assignments (r):
                            r.val
index0  index1                   
AUS_BOS Airbus A319-100         0
        Airbus A320-200 A       0
        Airbus A320-200 B       0
        Boeing 737 MAX 200      0
AUS_BUR Airbus A319-100         0
...                           ...
SRQ_IAD Boeing 737 MAX 200      0
VPS_BWI Airbus A319-100         0
        Airbus A320-200 A       0
        Airbus A320-200 B       0
        Boeing 737 MAX 200      0

[808 rows x 1 columns]

Total Profit: 0.0


In [120]:
# Debugging: Print parameter values for a specific route
print("Debugging route parameters:")
for route in ["AUS_IAD", "BOS_ORF"]:  # Example routes
    print(f"Route: {route}")
    print(f"a: {df.loc[df['airport_1_2'] == route, 'a'].values[0]}")
    print(f"b: {df.loc[df['airport_1_2'] == route, 'b'].values[0]}")
    print(f"y: {df.loc[df['airport_1_2'] == route, 'year_coeff'].values[0]}")
    print(f"c: {df.loc[df['airport_1_2'] == route, 'cost'].values[0]}")
    print(f"N: {df.loc[df['airport_1_2'] == route, 'predicted_passengers'].values[0]}")
    print()

Debugging route parameters:
Route: AUS_IAD
a: 353.0795727
b: 0.000287919
y: -0.009039018
c: 234.94394
N: 323.7849426269531

Route: BOS_ORF
a: 215.8656962
b: 0.000174883
y: 0.010823479
c: 85.25909
N: 243.7200164794922



In [121]:

# Check constraints and parameters in AMPL
%%ampl_eval
display a, b, y, c, N, C, R, t;

SyntaxError: invalid syntax (3262428469.py, line 3)